In [1]:
import numpy as np
from nltk.corpus import stopwords
#nltk.download('stopwords')
from nltk.tokenize import word_tokenize

### Levenshtein Distance

Levenshtein Distance clearly explained in https://www.youtube.com/watch?v=MiqoA-yF-0M. 

The difference betwen 2 strings are calculated based on number of edit operations needed to change 1 string to other.

Posible Edit Operations: 1) remove 2) replace and 3) insert

In [2]:
# Base Function

# Function calculate Levenshtein Distance between 2 given strings
# input: String 1, String 2
# output: [no of edits, difference ratio]

def check_equality(str1, str2):
    rows = len(str1)
    cols = len(str2)
    
    #Create a matrix to hold no. of edits for each character of 2 given strings
    distance = np.zeros((rows, cols), dtype=int)
    
    #Always the first character of both the string are empty, 
    #so number of edit values in first row and first column will be the index of that particular cell
    for i in range(1, rows):
        distance[i][0] = i
    for j in range(1, cols):
        distance[0][j] = j
    
    #Interate over all the character in the position of matrices, to compute the cost of edit operations
    for i in range(1, rows):
        for j in range(1, cols):
            ch1 = str1[i-1]
            ch2 = str2[j-1]
            if ch1 == ch2:
                cost = 0
            else:
                cost = 1
            #get the edit cost of the previous edits around this cell
            cost1 = distance[i][j-1]
            cost2 = distance[i-1][j-1]
            cost3 = distance[i-1][j]
            distance[i][j] = min(cost1, cost2, cost3)+cost
            
    #No of edits
    no_of_edits = distance[rows-1][cols-1]
    #Ratio of change
    total_len = rows+cols
    ratio = round((total_len-no_of_edits)*100/total_len, 2)

    return [no_of_edits, ratio]

In [3]:
[no_of_edits, ratio] = check_equality('Silvester Stalin Bruno Jr','Silvester Stalin Bruno Sr')
print('The strings are {} edits away'.format(no_of_edits))
print('The string\'s equality is {:.2f}%'.format(ratio))

The strings are 1 edits away
The string's equality is 98.00%


In [3]:
# Function check a string's partial presence in another string
# input: String 1 (bigger), String 2 (substring)
# output: [no of edits, difference ratio, matched string]

def check_partial_equality(fullstr, substr):
    len1 = len(fullstr)
    len2 = len(substr)
    
    #initialize output
    out_ratio = 0    
    out_match_str = ''
    out_no_of_edits = 0
    
    #Get a piece of string from main string of substr's length
    for i in range(0, len1):
        if (i+len2) <= len1:            
            str1 = fullstr[i:i+len2]
            str2 = substr
            [no_of_edits, ratio] = check_equality(str1, str2)
            if (ratio == 100):
                out_ratio = ratio
                out_match_str = str1                
                break
            elif (ratio > out_ratio):
                out_ratio = ratio
                out_match_str = str1
                out_no_of_edits = no_of_edits
    
    return [no_of_edits, ratio, out_match_str]

In [63]:
[no_of_edits, ratio, match_str] = check_partial_equality('Silvester Stalin Bruno Iruthaiyaraj' , 'Irudayaraj')
print('The string\'s equality is {:.2f}%'.format(ratio), 'for ',match_str)

The string's equality is 75.00% for  Iruthaiyar


In [4]:
# Function check 2 strings' equality by tokenizing
# input: String 1, String 2
# output: [difference ratio]

def check_token_full_equality(str1, str2):
    token1 = str1.split(' ')
    token2 = str2.split(' ')   
    len1 = len(token1)
    len2 = len(token2)
    
    if len1 > len2:
         token2 = token2 + [' ' for w in range(len1-len2)]
    elif len2 > len1:
        token1 = token1 + [' ' for w in range(len2-len1)]
    
    size = len(token1)
    total_edits = [0 for i in range(size)]
    out_ratio = 0
    
    for i in range(0, size):
        for j in range(0, size):
            [no_of_edits, ratio] = check_equality(token1[i], token2[j])
            if (ratio == 100):
                total_edits[i] = no_of_edits                
                break
            elif (ratio > out_ratio):
                out_ratio = ratio
                total_edits[i] = no_of_edits
    
    output_ratio = round((len(str1)+len(str2)-sum(total_edits))*100/(len(str1)+len(str2)),2)
    return output_ratio

In [65]:
 check_token_full_equality('Silvester Stalin Bruno Iruthaiyaraj' , 'Silvester Bruno Stalin Irudayaraj')

95.59

In [5]:
# Utility Functions
# 1. Remove special characters
def remove_spl_chars(word):
    spl_chars = [';', ':', '!', "*", '.', ',', '$', '#', '%', '&', '?']
    for w in spl_chars:
        word = word.replace(w, '')
    return word

# 2. Remove stop words and return the tokens
def get_tokens(datastr):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(datastr)
    tokens = [w for w in word_tokens if not w in stop_words]
    return tokens

In [15]:
# Function check string 1's presence in another string by tokenizing
# input: String 1 (bigger), String 2 (substring)
# output: [no of edits, difference ratio, matched string]

def check_token_presence(fullstr, substr):
    len1 = len(fullstr)
    len2 = len(substr)
    
    #Data Cleanup - Converts to lowercase, removes spl chars, remove stopwords and tokenizing
    fullstr = remove_spl_chars(fullstr.lower())
    substr = remove_spl_chars(substr.lower())

    token1 = get_tokens(fullstr)
    token2 = get_tokens(substr)
    total_edits = [0 for i in range(len(substr))]
    out_ratio = 0
    
    for i in range(0, len(token2)):
        for j in range(0, len(token1)):
            [no_of_edits, ratio] = check_equality(token1[j], token2[i])
            if (ratio == 100):
                total_edits[i] = no_of_edits                
                break
            elif (ratio > out_ratio):
                out_ratio = ratio
                total_edits[i] = no_of_edits
            
    output_ratio = round((len2-sum(total_edits))*100/len2,2)
    return output_ratio

In [20]:
check_token_presence('I am Silvester Stalin Bruno. I work in RBS India as Tech Lead for 4 years','Tech Lead')

100.0

In [27]:
str1 = 'The NLTK library is one of the oldest and most commonly used Python libraries for Natural Language Processing. NLTK supports stop word removal, and you can find the list of stop words in the corpus module. To remove stop words from a sentence, you can divide your text into words and then remove the word if it exits in the list of stop words provided by NLTK'
list_of_keywords = ['Natural Language Processing','NLP','NLTK']
out_ratio = [0 for i in list_of_keywords]
for k in range(0,len(out_ratio)):
    out_ratio[k] = check_token_presence(str1,list_of_keywords[k])
out_ratio

[100.0, 66.67, 100.0]